In [1]:

# Install esm and other dependencies
! pip install esm
! pip install matplotlib




  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
     ---------------------------------------- 0.0/666.1 kB ? eta -:--:--
     ------------------------------------- 666.1/666.1 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.4.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.4.26-py3-none-any.whl.metadata (2.5 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 2.4/2.4 MB 23.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ------------

  DEPRECATION: Building 'zstd' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'zstd'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [9 lines of output]
      
       Need pkg-config to find system libzstd. Or we try bundled one.
      c:\Users\avdh3\OneDrive\Dokumente\GitHub\group04-team04\.conda\Lib\site-packages\setuptools\_distutils\dist.py:289: UserWarning: Unknown distribution option: 'test_suite'
        warnings.warn(msg)
      running bdist_wheel
      running build
      running build_ext
      building 'zstd' extension
      error: Microsoft 

^C


  Using cached numpy-2.2.6-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached pillow-11.2.1-cp311-cp311-win_amd64.whl.metadata (9.1 kB)
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 8.1/8.1 MB 49.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 63.7 MB/s eta 0:00:00
Using cached numpy-2.2.6-cp311-cp311-win_amd64.whl (12.9 MB)
Using cached pillow-11.2.1-cp311-cp311-win_amd64.whl (2.7 MB)

   ---------------------------------------- 0/8 [pyparsing]
   ---------------------------------------- 0/8 [pyparsing]
   ---------------------------------------- 0/8 [pyparsing]
   ---------------------------------------- 0/8 [pyparsing]
   ----- ---------------------------------- 1/8 [pillow]
   ----- ---------------------------------- 1/8 [pillow]
   ----- ---------------------------------- 1/8 [pillow]
   ----- ---------------------

In [ ]:
#Set up Forge client for ESM C
from getpass import getpass
from esm.sdk import client

model = client(
    model="esmc-300m-2024-12", url="https://forge.evolutionaryscale.ai", token='MoBiPauli'
)

In [ ]:

from concurrent.futures import ThreadPoolExecutor
from typing import Sequence

from esm.sdk.api import (
    ESM3InferenceClient,
    ESMProtein,
    ESMProteinError,
    LogitsConfig,
    LogitsOutput,
    ProteinType,
)

EMBEDDING_CONFIG = LogitsConfig(
    sequence=True, return_embeddings=True, return_hidden_states=True
)


def embed_sequence(model: ESM3InferenceClient, sequence: str) -> LogitsOutput:
    protein = ESMProtein(sequence=sequence)
    protein_tensor = model.encode(protein)
    output = model.logits(protein_tensor, EMBEDDING_CONFIG)
    return output


def batch_embed(
    model: ESM3InferenceClient, inputs: Sequence[ProteinType]
) -> Sequence[LogitsOutput]:
    """Forge supports auto-batching. So batch_embed() is as simple as running a collection
    of embed calls in parallel using asyncio.
    """
    with ThreadPoolExecutor() as executor:
        futures = [
            executor.submit(embed_sequence, model, protein) for protein in inputs
        ]
        results = []
        for future in futures:
            try:
                results.append(future.result())
            except Exception as e:
                results.append(ESMProteinError(500, str(e)))
    return results

In [ ]:
import pandas as pd
import numpy as np
from esm.sdk.api import ESM3InferenceClient, ESMProtein
# Eigene Datei laden
df = pd.read_csv("/Users/paulinekipp/Documents/GitHub/group04-team04/data/ab_ag_annotated_with_CDR_strings.tsv", sep="\t")
df = df.dropna(subset=["CDR_L3_string"])

# Modell initialisieren
model = ESM3InferenceClient()

# ESMProtein-Objekte erstellen
proteins = [ESMProtein(sequence=s) for s in df["CDR_L3_string"]]

# Embeddings berechnen
results = batch_embed(model, proteins)

# Nur erfolgreiche Ergebnisse behalten
valid_results = [r for r in results if hasattr(r, "embeddings")]

# Dann Mittelwert berechnen
df["embedding_avg"] = [np.mean(r.embeddings, axis=0) for r in valid_results]

ValueError: Length of values (0) does not match length of index (5254)

In [ ]:
#Quelle für ESMC
@misc{esm2024cambrian,
  author = {{ESM Team}},
  title = {ESM Cambrian: Revealing the mysteries of proteins with unsupervised learning},
  year = {2024},
  publisher = {EvolutionaryScale Website},
  url = {https://evolutionaryscale.ai/blog/esm-cambrian},
  urldate = {2024-12-04}
}

In [ ]:
#Visualisierung der Embeddings mit hilfe von PCA
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

#Mittelwert der Embeddings bestimmen (unnötig weil vorher schon gemacht?)
embedding_avg = [np.mean(r.embeddings, axis=0) for r in valid_results]
#in ein dataframe umwandeln
embedding_df = pd.DataFrame(embedding_avg)
#PCA
pca= PCA(n_component=2)
pca_result = pca.fit_transform(embedding_df)
#Plot preparen
plt.figure(figsize=(10,6))
plt.scatter(pca_result[:,0], pca_result[:,1], alpha=0.7)
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.title("PCA der CDR Embeddings (ESM C)")
plt.grid(True)
plt.tight_layout()
plt.show()